<a href="https://colab.research.google.com/github/ccg-esb/EvK/blob/main/py_pOXA48_SI5_communities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5. Computer experiments: multistrain communities

In [18]:
import numpy as np
import os
import csv
import sys
import matplotlib.pyplot as plt
import colorcet as cc
import time
import multiprocessing
import random
from tabulate import tabulate
import itertools
import os
import pickle


File structure

In [19]:

num_days = 10
num_experiments = 3
str_E = 'gaussian_noise'
iEs = [0,1,2]


In [20]:

# Define your paths
root='/content/drive/MyDrive/SYNC_Projects/pOXA48/2023/code/py-files/'

path='./'
figPath = path+'figures/'
envPath = path+'env/'
dataPath = path+'data/'


strains_subsetE = [1, 2, 6, 9, 11, 15, 18, 20, 21, 24]
strains_subsetK = [25, 26, 29, 34, 37, 38, 39, 41, 43, 45]

strains_subset = strains_subsetE + strains_subsetK

print("subset: ",strains_subset)

strains = ['C001', 'C002',  'C006',  'C011',  'C012',  'C021',  'C022',  'C031',  'C051',  'C063',  'C094',  'C107',  'C115',  'C131',  'C141',  'C201',  'C227',  'C232',  'C247',  'C261',  'C286',  'C290',  'C302',  'C309',  'C324',  'K037',  'K038',  'K087',  'K094',  'K112',  'K114',  'K125',  'K141',  'K168',  'K177',  'K200',  'K201',  'K209',  'K213',  'K216',  'K224',  'K225',  'K241',  'K248',  'K249',  'K253',  'K257',  'K275',  'K285',  'K300']
plasmids = ['WT','TC']

tot_strains = int(len(strains))
cmap_strains = cc.glasbey_light[:tot_strains]


subset:  [1, 2, 6, 9, 11, 15, 18, 20, 21, 24, 25, 26, 29, 34, 37, 38, 39, 41, 43, 45]


In [21]:
from google.colab import drive
drive.mount('/content/drive')

os.chdir(root)

sys.path.insert(0, 'src/')
from pOXA48_S1 import *
from pOXA48_S2 import *
from pOXA48_S3 import *

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Model parameters

In [22]:

B0 = 1e6 #Initial bacterial density
T = 24 #Duration of experimental season
S0 = 1 #Concentration of imiting resource
extinction_threshold=1 #Extinction threshold
alphas=[1e-10, 1e-12] #Antibiotic degradation rate
d=0.1 #Transfer dilution rate

A_max=65536*2 #Maximum antibiotic concentrations=[32768, 256, 1024, 32]

expe_params = {
    'B0': B0, #Initial bacterial density
    'A_max': A_max, #Maximum drug concentration
    'alphas': np.array(alphas), #Antibiotic degradation rate
    'T': T,  # Length of experiment
    'S0': S0,  # Resource concentration
    'd': d,  # Resource concentration
    'extinction_threshold': extinction_threshold,
}


## Load experimental parameters from files
(MCMC & Resistance & Conjugation permissiveness & MIC)

In [23]:
model_params = import_model_params("%smodel_params.csv"%(dataPath), expe_params)

#display_model_params_stats(model_params)

## Computer simulations: Serial transfers (Multiple Es)

In [24]:
def simulate_environment_multistrain(model_params, istrains, E):
    #print('.', end="", flush=True)

    lbl_E = ''  # '%s_%s' % (str_E, indx_E)

    # Simulate transfers
    times_list, ys_list, strains_params_list = simulateTransfers_multistrain(model_params, istrains, E)

    # Get final points
    final_times, final_ys = get_final_points(times_list, ys_list)

    # Analyze simulation results
    Btot, BpE, BpK, freqpE, freqpK = analyze_simulation(model_params, istrains, final_ys)

    return Btot, BpE, BpK, freqpE, freqpK


def simulate_environments_multistrain(model_params, istrains, Es):
    start_time = time.time()  # Record the start time
    freqpEs = []
    freqpKs = []
    BpEs=[]
    BpKs=[]
    Btot=[]
    for E in Es:
        Btot, BpE, BpK, freqpE, freqpK = simulate_environment_multistrain(model_params, istrains, E)
        freqpEs.append(freqpE)
        freqpKs.append(freqpK)
        BpEs.append(BpE)
        BpKs.append(BpK)
        Btot.append(Btot)

    end_time = time.time()  # Record the end time
    elapsed_time = end_time - start_time  # Calculate the elapsed time

    print("Total time elapsed: {:.2f} seconds".format(elapsed_time))
    return Btot, BpEs, BpKs, freqpEs, freqpKs


Here we run the simulations:

In [25]:



def load_simulation_results(filename):
    with open(filename, "rb") as f:
        results = pickle.load(f)
    return results["istrains"], results["Es"],  results["Btot"], results["BpEs"], results["BpKs"], results["freqpEs"], results["freqpKs"]


def save_simulation_results(filename, istrains, Es, Btot, BpEs, BpKs, freqpEs, freqpKs):
    results = {
        "istrains": istrains,
        "Es": Es,
        "Btot": Btot,
        "BpEs": BpEs,
        "BpKs": BpKs,
        "freqpEs": freqpEs,
        "freqpKs": freqpKs,
    }
    with open(filename, "wb") as f:
        pickle.dump(results, f)




In [26]:


#max_strains = len(strains_subset)
#istrains = [0, 2, 4] #example
#print('istrains=', istrains)

#str_istrains = [str(x) for x in istrains]  # Convert all elements to string
#sim_lbl = "_".join(str_istrains)  # Join elements with "_"
#fileName = "sim_Es_%s.pkl" % sim_lbl  # Construct the filename

#if os.path.isfile("%s%s"%(runPath, fileName)):
#  istrains, Es, freqps, Bps, Bfs = load_simulation_results("%s%s"%(runPath, fileName))
#  print("Loading %s"%(fileName))
#else:
#  Bps, Bfs, freqps = simulate_environments_multistrain(model_params, istrains, Es)
#  save_simulation_results("%s%s"%(runPath, fileName), istrains, Es, freqps, Bps, Bfs)
#  print("Saving %s"%(fileName))

In [27]:
#print('\t\tistrains=%s'%istrains)
#print('\t\tEs=%s'%Es)
#print('\t\tfreqpT[-1]=%s' % freqps)
#print('\t\tBpT[-1]=%s' % Bps)
#print('\t\tBfT[-1]=%s' % Bfs)

In [28]:

def get_sequences_from_files(directory, length=None):
    # Initialize a list to hold the sequences
    sequences = []

    # Loop over each file in the directory
    for filename in os.listdir(directory):
        # Check if the file matches the desired format
        if filename.startswith("sim_Es_") and filename.endswith(".pkl"):
            # Extract the sequence from the filename
            sequence = filename[len("sim_Es_"):-len(".pkl")]

            # Convert sequence string to list of integers
            sequence = list(map(int, sequence.split('_')))

            # Only add to list if no length was specified or the sequence has the correct length
            if length is None or len(sequence) == length:
                sequences.append(sequence)

    return sequences


# Use the function


## Parallel computer simulations: Serial transfers (Multiple Es, Multiple strains)

In [29]:

def simulate_subset_parallel(model_params, istrains=[], Es=[]):
    freqpEs_all=[]
    freqpKs_all=[]
    Btots_all=[]
    BpEs_all=[]
    BpKs_all=[]
    istrains_all=[]
    Es_all=[]


    for subset_length in range(1, len(istrains) + 1):
      freqpEs = []
      freqpKs = []
      Btots=[]
      BpEs=[]
      BpKs=[]

      subset = istrains[:subset_length]
      print(f"\t{subset}: ", end="", flush=True)

      str_subset = [str(x) for x in subset]  # Convert all elements to string
      sim_lbl = "_".join(str_subset)  # Join elements with "_"
      fileName = "sim_Es_%s.pkl" % sim_lbl  # Construct the filename

      if os.path.isfile("%s%s"%(runPath, fileName)):

        istrains_subset, Es, Btot, BpEs, BpKs, freqpEs, freqpKs = load_simulation_results("%s%s"%(runPath, fileName))
        print(" Loading %s%s"%(runPath, fileName))
      else:
        # Create a multiprocessing Pool with the number of desired processes
        pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())  # Use all available CPU cores

        # Iterate over each E in Es
        for E in Es:

              # Run the simulation for the current E and subset
              this_Btot, this_BpEs, this_BpKs, this_freqpEs, this_freqpKs = simulate_environment_multistrain(model_params, subset, E)
              Btots.append(this_Btot)
              BpEs.append(this_BpEs)
              BpKs.append(this_BpKs)
              freqpEs.append(this_freqpEs)
              freqpKs.append(this_freqpKs)

        save_simulation_results("%s%s"%(runPath, fileName), subset, Es, Btots, BpEs, BpKs, freqpEs, freqpKs) #here
        print(" Saving %s%s"%(runPath,fileName))

      istrains_all.append(subset)
      Btots_all.append(Btots)
      Es_all.append(Es)
      BpEs_all.append(BpEs)
      BpKs_all.append(BpKs)
      freqpEs_all.append(freqpEs)

    return istrains_all, Es_all, Btots_all, BpEs_all, BpKs_all, freqpEs_all, freqpKs_all


## Community combinatorics

In [30]:
def select_random_elements(strains_subset, num_elements):
    random_elements = random.sample(strains_subset, num_elements)
    return random_elements

In [31]:


def run_experiment(model_params, strains_subset, num_experiments, istrains_done):

    max_strains = len(strains_subset)

    istrains_experiments = []
    Es_experiments = []
    Btots_experiments = []
    BpEs_experiments = []
    BpKs_experiments = []
    freqpEs_experiments = []
    freqpKs_experiments = []

    for i in range(num_experiments+1):
        start_time = time.time()  # Record the start time

        if i < len(istrains_done):
            istrains = istrains_done[i]
            print('* ', i, ': istrains =', istrains)
        else:
            istrains = select_random_elements(strains_subset, max_strains)
            print('** ', i, ': istrains =', istrains)

        # Run the simulations in parallel
        istrains_subset, Es_subset, Btots_subset, BpEs_subset, BpKs_subset, freqpEs_subset, freqpKs_subset = simulate_subset_parallel(model_params, istrains, Es)

        istrains_experiments.append(istrains_subset)
        Es_experiments.append(Es_subset)
        Btots_experiments.append(Btots_subset)
        BpEs_experiments.append(BpEs_subset)
        BpKs_experiments.append(BpKs_subset)
        freqpEs_experiments.append(freqpEs_subset)
        freqpKs_experiments.append(freqpKs_subset)

        end_time = time.time()  # Record the end time
        elapsed_time = end_time - start_time  # Calculate the elapsed time

        print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

    return istrains_experiments, Es_experiments, Btots_experiments, BpEs_experiments, BpKs_experiments, freqpEs_experiments, freqpKs_experiments



In [32]:


def plot_freqps_experiments(freqpEs_experiments, freqpKs_experiments):
    # Number of experiments
    num_experiments = len(freqpEs_experiments)

    fig, ax = plt.subplots(1, 1, figsize=(6,4))

    # Get the default color cycle
    color_cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']

    # Define standard deviation for jitter
    jitter_sd = 0.1

    # Loop over each experiment
    for n in range(num_experiments):

        num_subsets=len(freqpEs_experiments[n])

        for s in range(num_subsets):

            num_members=len(istrains_experiments[n][s])

            num_environments = len(freqpEs_experiments[n][s])

            for e in range(num_environments):

                # Use color from cycle, looping back to the start if there are more experiments than colors
                color = color_cycle[n % len(color_cycle)]


                freqpEs=(freqpEs_experiments[n][s][e])
                freqpKs=(freqpKs_experiments[n][s][e])

                # Add jitter
                jitter = np.random.normal(scale=jitter_sd)

                ax.plot(num_members + jitter, freqpEs[-1], 'o', color=color, alpha=0.25)
                ax.plot(num_members + jitter, freqpKs[-1], '*', color=color, alpha=0.25)

    ax.set_xlabel('Number of strains')
    ax.set_ylabel('Plasmid frequency (%)')
    ax.set_ylim([-0.05,1.05])

    plt.show()







In [33]:
def get_species_from_istrains(model_params, istrains):
    # Initialize the list to store the types
    species = []

    # Loop through each element in istrains
    for istrain in istrains:
        # Find the corresponding type in model_params and append it to the list
        this_species = model_params.loc[istrain, 'specie']
        species.append(this_species)

    # Return the list of types
    return species


def analyze_simulation(model_params, this_istrains, final_ys):

    B = np.array(final_ys)
    B=B[:,2:]
    num_days = B.shape[0]
    num_strains = B.shape[1] // 2


    species=get_species_from_istrains(model_params, this_istrains)

#    num_strains = len(species)

    # Iterate over strains and add their populations to the relevant counters
    Btot=np.zeros(num_days)
    BpE=np.zeros(num_days)
    BpK=np.zeros(num_days)
    BfE=np.zeros(num_days)
    BfK=np.zeros(num_days)
    freqpE=np.zeros(num_days)
    freqpK=np.zeros(num_days)
    for day in range(num_days):

      #print("B[",day,"]=",B[day,:])

      for i in range(num_strains):
        if species[i] == 'E':
            BpE[day] += B[day, i]
            BfE[day] += B[day, i + num_strains]
        elif species[i] == 'K':
            BpK[day] += B[day, i]
            BfK[day] += B[day, i + num_strains]

        Btot[day]+=B[day, i + num_strains]+B[day, i]

      # Compute frequencies for E and K strains separately
      if Btot[day] > 0:
          freqpE[day] = BpE[day] / Btot[day]
          freqpK[day] = BpK[day] / Btot[day]
      else:
          freqpE[day] = np.nan
          freqpK[day] = np.nan

    #print("\n\t\tBtot=",Btot)
    #print("\t\tBpE=",BpE)
    #print("\t\tBpK=",BpK)
    #print("\t\tBfE=",BfE)
    #print("\t\tBfK=",BfK)
    #print("\t\tfreqpE=",freqpE)
    #print("\t\tfreqpK=",freqpK)

    return Btot, BpE, BpK, freqpE, freqpK


In [34]:

Es_norm=load_environments(str_E, envPath, num_days, iEs)

powers = np.arange(0, 8)  # array of powers of two exponents
Amax_values=np.power(2, powers)  # calculate 2 raised to each exponent

for Amax in Amax_values:
  print("******** Amax=%s"%Amax)

  Es = [Amax * e for e in Es_norm]

  expeLabel='Gaussian_N%s_A%s'%(num_days, int(Amax))
  runPath = path+'runs/'+expeLabel+'/'
  print("runPath=%s"%runPath)


  if not os.path.exists(runPath):
    os.makedirs(runPath)

  istrains_done = get_sequences_from_files(runPath, length=len(strains_subset))
  istrains_done = istrains_done[:num_experiments]

  istrains_experiments, Es_experiments, Btots_experiments, BpEs_experiments, BpKs_experiments, freqpEs_experiments, freqpKs_experiments = run_experiment(model_params, strains_subset, num_experiments, istrains_done)
  #plot_freqps_experiments(freqpEs_experiments, freqpKs_experiments)

******** Amax=1
runPath=./runs/Gaussian_N10_A1/
*  0 : istrains = [39, 43, 34, 29, 45, 20, 41, 26, 38, 24, 37, 1, 11, 2, 21, 6, 15, 25, 9, 18]
	[39]:  Loading ./runs/Gaussian_N10_A1/sim_Es_39.pkl
	[39, 43]:  Loading ./runs/Gaussian_N10_A1/sim_Es_39_43.pkl
	[39, 43, 34]:  Loading ./runs/Gaussian_N10_A1/sim_Es_39_43_34.pkl
	[39, 43, 34, 29]:  Loading ./runs/Gaussian_N10_A1/sim_Es_39_43_34_29.pkl
	[39, 43, 34, 29, 45]:  Loading ./runs/Gaussian_N10_A1/sim_Es_39_43_34_29_45.pkl
	[39, 43, 34, 29, 45, 20]:  Loading ./runs/Gaussian_N10_A1/sim_Es_39_43_34_29_45_20.pkl
	[39, 43, 34, 29, 45, 20, 41]:  Loading ./runs/Gaussian_N10_A1/sim_Es_39_43_34_29_45_20_41.pkl
	[39, 43, 34, 29, 45, 20, 41, 26]:  Loading ./runs/Gaussian_N10_A1/sim_Es_39_43_34_29_45_20_41_26.pkl
	[39, 43, 34, 29, 45, 20, 41, 26, 38]:  Loading ./runs/Gaussian_N10_A1/sim_Es_39_43_34_29_45_20_41_26_38.pkl
	[39, 43, 34, 29, 45, 20, 41, 26, 38, 24]:  Loading ./runs/Gaussian_N10_A1/sim_Es_39_43_34_29_45_20_41_26_38_24.pkl
	[39, 43, 34